In [0]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D, BatchNormalization, Activation
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.preprocessing.image import img_to_array, load_img
import pandas as pd
import numpy as np
import cv2
import os
import glob
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

import tensorflow as tf

from keras import backend as K
from keras.applications import inception_v3
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_v3 import preprocess_input as inception_v3_preprocessor
from keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator


from keras.models import load_model

from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import to_categorical

import math
from tqdm import tqdm
K.clear_session()

In [5]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
images_links = glob.glob('/gdrive/My Drive/Data/DogBreed/train/*')

In [0]:
# Training parameters
img_width, img_height = 299, 299
train_data_dir = '/gdrive/My Drive/Data/DogBreed/train/'
nb_classes = 120

df_train = pd.read_csv('/gdrive/My Drive/Data/DogBreed/labels.csv')

# Get the labels of the dogs breed
target_labels = df_train['breed']

In [0]:
df_train = pd.get_dummies(df_train, prefix='breed', columns=['breed'])

In [0]:
output_col = df_train.columns
output_col = output_col.drop(['id'])

In [12]:
# Create train generator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   horizontal_flip = 'true',
                                   validation_split=0.2)


train_generator = train_datagen.flow_from_dataframe(df_train, 
                                                    directory=train_data_dir, 
                                                    x_col='id', 
                                                    y_col=output_col, 
                                                    has_ext=False, 
                                                    target_size=(299, 299), 
                                                    class_mode='other', 
                                                    batch_size=10, 
                                                    shuffle=False, 
                                                    seed=10,
                                                    subset="training")

test_generator = train_datagen.flow_from_dataframe(df_train, 
                                                  directory=train_data_dir, 
                                                  x_col='id', 
                                                  y_col=output_col, 
                                                  has_ext=False, 
                                                  target_size=(299, 299), 
                                                  class_mode='other', 
                                                  batch_size=10, 
                                                  shuffle=False, 
                                                  seed=10,
                                                  subset="validation")

Found 8178 images.
Found 2044 images.


In [13]:
# Get the InceptionV3 model so we can do transfer learning
base_model = InceptionV3(weights = 'imagenet', 
                         include_top = False, 
                         input_shape=(299, 299, 3))

87916544/87910968 [==============================] - 3s 0us/step


In [0]:
# Add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)

# Add a fully connected layer
x = Dense(512)(x)
x = BatchNormalization()(x)
x = Activation('relu')(x)
predictions = Dense(nb_classes, activation='softmax')(x)

# The model we will train
model = Model(inputs = base_model.input, outputs = predictions)
    
#for layer in base_model.layers[:249]:
#   layer.trainable = False
#for layer in base_model.layers[249:]:
#   layer.trainable = True

for layer in base_model.layers:
  layer.trainable = False
  
    
# Compile with Adam
model.compile(Adam(lr=.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [24]:
from tensorboardcolab import TensorBoardColab, TensorBoardColabCallback
tbc=TensorBoardColab()

Wait for 8 seconds...
TensorBoard link:
https://c7ed5bd6.ngrok.io


In [0]:
# Train the model
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=test_generator.n//test_generator.batch_size

tbcallback = TensorBoard(log_dir='/gdrive/My Drive/Data/DogBreed/Graphs',  
                         histogram_freq=0, 
                         write_graph=True, 
                         write_images=True)

earlyStopping = EarlyStopping(monitor='val_loss', 
                              patience=10, 
                              verbose=0, 
                              mode='min')

mcp_save = ModelCheckpoint('/gdrive/My Drive/Data/DogBreed/Models/dogbreeds_model_v7.h5', 
                           save_best_only=True, 
                           monitor='val_loss', 
                           mode='min')

model.fit_generator(train_generator,
                    steps_per_epoch = STEP_SIZE_TRAIN,
                    validation_steps = STEP_SIZE_VALID,
                    validation_data = test_generator,
                    epochs = 25,
                    verbose = 1,
                    callbacks = [TensorBoardColabCallback(tbc), mcp_save, tbcallback])

Epoch 1/25
817/817 [==============================] - 2576s 3s/step - loss: 2.6991 - acc: 0.4371 - val_loss: 0.8477 - val_acc: 0.7716
Epoch 2/25
817/817 [==============================] - 312s 381ms/step - loss: 1.3632 - acc: 0.6767 - val_loss: 0.7000 - val_acc: 0.7901
Epoch 3/25
817/817 [==============================] - 308s 377ms/step - loss: 1.0848 - acc: 0.7269 - val_loss: 0.6735 - val_acc: 0.8019
Epoch 4/25
817/817 [==============================] - 308s 377ms/step - loss: 0.9175 - acc: 0.7612 - val_loss: 0.6768 - val_acc: 0.8004
Epoch 5/25
817/817 [==============================] - 306s 374ms/step - loss: 0.8223 - acc: 0.7887 - val_loss: 0.6662 - val_acc: 0.7920
Epoch 6/25
817/817 [==============================] - 308s 377ms/step - loss: 0.7076 - acc: 0.8147 - val_loss: 0.6781 - val_acc: 0.8073
Epoch 7/25
817/817 [==============================] - 309s 378ms/step - loss: 0.6425 - acc: 0.8315 - val_loss: 0.6846 - val_acc: 0.8073
Epoch 8/25
817/817 [==============================

In [0]:
saved_model = load

In [17]:
!pip install keras_metrics

In [15]:
!pip install tensorboardcolab

  Running setup.py bdist_wheel for tensorboardcolab ... - done
  Stored in directory: /root/.cache/pip/wheels/ab/74/02/cda602d1dc28b2f12eab313c49b9bfa14d6371326bc2590e06
Successfully built tensorboardcolab


In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

['/job:localhost/replica:0/task:0/device:GPU:0']

In [0]:
# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/17/0f/04/b79c006972335e35472c0b835ed52bfc0815258d409f560108
Successfully built gputil
  Running setup.py bdist_wheel for humanize ... - done
  Stored in directory: /root/.cache/pip/wheels/69/86/6c/f8b8593bc273ec4b0c653d3827f7482bb2001a2781a73b7f44
Successfully built humanize
Gen RAM Free: 11.3 GB  | Proc size: 2.3 GB
GPU RAM Free: 3046MB | Used: 8395MB | Util  73% | Total 11441MB


In [0]:
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

Gen RAM Free: 11.3 GB  | Proc size: 2.3 GB
GPU RAM Free: 3046MB | Used: 8395MB | Util  73% | Total 11441MB


In [0]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 149, 149, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 149, 149, 32) 96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 149, 149, 32) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (